# General Slides

In [23]:
#import plotly.graph_objects as go
#import chart_studio
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np

# Import pptx libraries
import pptx
from pptx import Presentation
from pptx.chart.data import CategoryChartData
from pptx.enum.chart import XL_CHART_TYPE
from pptx.util import Inches

from pptx.dml.color import RGBColor
from pptx.enum.dml import MSO_THEME_COLOR
from pptx.util import Pt

from pptx.enum.text import PP_ALIGN

import pptx


# Display max columns when viewing data
pd.set_option('display.max_columns', 500)

In [24]:
msft = pd.read_excel('/Users/brandongoldney/Documents/Projects/Python_PPTX/MSFT_Financials.xls', index_col=False)

# Remove any unnamed columns
cols = [string for string in msft.columns if string.lower()[:7] != 'unnamed']
msft = msft[cols]

# Set rows as the index
msft = msft.set_index(['Account'])

#Convert any columns inported as "object" to "float32"
cols = msft.select_dtypes(exclude=['float']).columns
msft[cols] = msft[cols].apply(pd.to_numeric, downcast='float', errors='coerce')

# Drop any columns with nan values
msft = msft.dropna(axis=1, how = 'any')

msft.head()


,Q1 15,Q2 15,Q3 15,Q4 15,Q1 16,Q2 16,Q3 16,Q4 16,Q1 17,Q2 17,Q3 17,Q4 17,Q1 18,Q2 18,Q3 18,Q4 18,Q1 19,Q2 19,Q3 19
Account,,,,,,,,,,,,,,,,,,,
Total Revenue,21729.00,22180.0,20379.00,23796.00,20531.00,20614.0,21928.00,25826.0,23212.0,25605.00,24538.00,28918.00,26819.00,30085.00,29084.00,32471.00,30571.00,33717.0,33055.00
Gross Profit,14568.00,14712.0,13172.00,13924.00,12809.00,12635.0,14084.00,15925.0,15152.0,17560.00,16260.00,17854.00,17550.00,20343.00,19179.00,20048.00,20401.00,23305.0,22649.00
Operating Income,6784.00,6385.0,5793.00,6026.00,5283.00,4190.0,6715.00,7905.0,6723.0,10279.00,7708.00,8679.00,8292.00,10379.00,9955.00,10258.00,10341.00,12405.0,12686.00
Net Income,4985.00,-3195.0,4902.00,5018.00,3756.00,3122.0,5667.00,6267.0,5486.0,8069.00,6576.00,-6302.00,7424.00,8873.00,8824.00,8420.00,8809.00,13187.0,10678.00
EPS,0.61,-0.4,0.61,0.62,0.47,0.4,0.72,0.8,0.7,1.03,0.84,-0.82,0.95,1.14,1.14,1.08,1.14,1.7,1.38


# Load Presentation as Object

In [25]:
# This loads the MASTER presentation
prs = Presentation('General_Slides - MASTER.pptx')

In [26]:
# The code below shows the idx values for each placeholder
slide = prs.slides[1]

for shape in slide.shapes:
     if shape.is_placeholder:
         phf = shape.placeholder_format
         print('%d, %s' % (phf.idx, phf.type))

0, TITLE (1)
4, SLIDE_NUMBER (13)
15, CHART (8)


**Slide 1: Leverage**

In [27]:
# The code below is needed to access the chart on the second slide (i.e. prs.slides[1])
slide = prs.slides[1] 
chart = slide.shapes[2].chart # Not sure why choosing shapes[1] works but it does work

chart_data = CategoryChartData()

In [28]:
# Create window for lookback period - this determines how many historical periods are included (i.e. -4 means 4 historical periods)
window = -12

# Create variables to be place into chart
debt_ebitda = msft.loc['Total Debt / EBITDA'].iloc[window:]
lev_limit = msft.loc['Leverage Limit'].iloc[window:]
equity_assets = msft.loc['Equity / Total Assets'].iloc[window:]

# Create dates
dates_str = msft.columns.astype(str)
dates_str = dates_str[window:]

# Create chart_data
chart_data = CategoryChartData()

# This updates the dates (i.e. categories)
chart_data.categories = dates_str

# Update the actual data
chart_data.add_series('Total Debt / EBITDA', (debt_ebitda))
chart_data.add_series('Leverage Limit', lev_limit)
chart_data.add_series('Equity / Total Assets', equity_assets)

# Replace existing data
chart.replace_data(chart_data)
chart.has_major_gridlines = False

**Slide 4: Composition of Operating Cash Flow**

In [29]:
# The code below shows the idx values for each placeholder
slide = prs.slides[3]

for shape in slide.shapes:
     if shape.is_placeholder:
         phf = shape.placeholder_format
         print('%d, %s' % (phf.idx, phf.type))

0, TITLE (1)
1, OBJECT (7)
4, SLIDE_NUMBER (13)


In [30]:
# The code below is needed to access the chart on the third slide (i.e. prs.slides[2])
slide = prs.slides[3]
chart = slide.shapes[1].chart

chart_data = CategoryChartData()

In [31]:
# Create window for lookback period - this determines how many historical periods are included (i.e. -4 means 4 historical periods)
window = -7

# Create variables to be place into chart
net_income = msft.loc['Net Income'].iloc[window:]
dep_amort = msft.loc['Depreciation & Amort.'].iloc[window:]
non_cash = msft.loc['Other Non-Cash'].iloc[window:]
delta_wc = msft.loc['Change in Working Capital'].iloc[window:]
cfo = msft.loc['Cash from Operations'].iloc[window:]


# Create date
date = pd.Series(msft.columns[-7:])

In [32]:
# Create chart_data
chart_data = CategoryChartData()

# This updates the dates (i.e. categories)
chart_data.categories = date

# Update the actual data
chart_data.add_series('Net Income - slide', net_income)
chart_data.add_series('Depreciation & Amort.', dep_amort)
chart_data.add_series('Other Non-Cash', non_cash)
chart_data.add_series('Change in Operating Act.', delta_wc)
chart_data.add_series('Operating Cash Flow', cfo)

# Replace existing data
chart.replace_data(chart_data)
chart.has_major_gridlines = False

**Slide 6: Table**

In [33]:
# The code below shows the idx values for each placeholder
slide = prs.slides[5]
for shape in slide.placeholders:
     print('%d, %s' % (shape.placeholder_format.idx, shape.name))

0, Title 2
1, Content Placeholder 4
4, Slide Number Placeholder 3


In [34]:
# Edit the sixth slide
slide = prs.slides[5]

# Update Slide Title
title_placeholder = slide.shapes.title
title_placeholder.text = 'Key Metrics' # the variable slide_title is defined above in the "Create Dynamic Text" section

# Format Slide Title
text_frame = title_placeholder.text_frame
p = text_frame.paragraphs[0]
run = p.add_run()
font = run.font
font.name = 'Arial'
font.size = Pt(18)
font.bold = False
font.color.rgb = RGBColor(109,109,109)

**Create data for table**

In [35]:
# This variable will determine how many periods of data are shown.  "-20" means select the last 20 rows of data. 
# In other words, select today minus 20 quarters. 

# NOTE: window does not have a negative before the 8 because the negative will mess up the for loops below
window = 8

# Create variables for chart data
ebitda = msft.loc['EBITDA',:][window:-2]
levered_fcf = msft.loc['Levered Free Cash Flow',:][window:-2]
leverage = msft.loc['Leverage',:][window:-2]

# Round variables to two digits
leverage = round(leverage,0)
levered_fcf = round(levered_fcf,0)



In [36]:
# Convert to strings because python_pptx tables can only handle strings
ebitda = [str(i) for i in ebitda]
levered_fcf = [str(i) for i in levered_fcf]
leverage = [str(i) for i in leverage]

# Add x to the end of every item in leverage
leverage = [leverage + 'x' for leverage in leverage]
#leverage

In [37]:
# Add comma after every third number
ebitda2 = []
for i in range(0, len(ebitda)):
    ebitda2.append("{:,}".format(float(ebitda[i])))

# Add comma after every third number
levered_fcf2 = []
for i in range(0, len(ebitda)):
    levered_fcf2.append("{:,}".format(float(ebitda[i])))

In [38]:
# Create dates
table_dates = msft.columns[-9:].tolist() # 9 instead of 8 because need to add the '$ Millions' in

In [39]:
# Remove the ".0" from each entry in ebitda
ebitda2 = [ebitda2[:-2] for ebitda2 in ebitda2]
# Remove the ".0" from each entry in leveraged_fcf
levered_fcf2 = [levered_fcf2[:-2] for levered_fcf2 in levered_fcf2]

In [40]:
# Create table object - this is needed to add the data to the table
table = slide.shapes[2].table
table_data = CategoryChartData()

In [41]:
for i in range(0, window):
        table.cell(0,i+1).text = table_dates[i] # start at column i + 1 to avoid writing over the row names
        
for i in range(0, window):
        table.cell(1,i+1).text = ebitda2[i]
        font.size = Pt(10)

for i in range(0, window):
        table.cell(2,i+1).text = levered_fcf2[i]
        font.size = Pt(10)

for i in range(0, window):
        table.cell(3,i+1).text = leverage[i]
        font.size = Pt(10)

# Save Presentation

In [42]:
prs.save('General_Slides - OUTPUT.pptx')

**Slide 4: Capital Plan Recommendation: Graph of Equity Book Value, LT Debt, and ST Debt**

In [ ]:
# The code below is needed to access the chart on the second slide (i.e. prs.slides[1])
slide = prs.slides[1]
chart = slide.shapes[1].chart

chart_data = CategoryChartData()


In [ ]:
# The code below is needed to access the chart on the fourth slide (i.e. prs.slides[3])
slide = prs.slides[3]
chart = slide.shapes[0].chart

chart_data = CategoryChartData()

In [ ]:
# Create window for lookback period - this determines how many historical periods are included (i.e. -4 means 4 historical periods)
window = -19

# Create data for each variable 
# Historical actual values
equity_bv = adm.loc['Equity (book value)'].iloc[window:] # NOTE, THE NEGATIVE SIGN BEFORE WINDOW. See note above
lt_debt = adm.loc['LT Debt'].iloc[window:]
st_debt = adm.loc['ST Debt'].iloc[window:]

# Drop the NaNs
equity_bv = equity_bv[~np.isnan(equity_bv)].tolist()
lt_debt = lt_debt[~np.isnan(lt_debt)].tolist()
st_debt = st_debt[~np.isnan(st_debt)].tolist()

# Update lookback period
window = -1

# Plan Numbers
plan_equity = adm.loc['Plan: Equity (book value)'].iloc[window:]
plan_lt_debt = adm.loc['Plan: LT Debt'].iloc[window:].tolist()
plan_st_debt = adm.loc['Plan: ST Debt'].iloc[window:].tolist()

# Next Year: Low EPS Scenarios
next_yr_low_bv = adm.loc['Next Year Low: BV'].iloc[window:]
next_yr_low_lt_debt = adm.loc['Next Year Low: LT Debt'].iloc[window:].tolist()
next_yr_low_st_debt = adm.loc['Next Year Low: ST Debt'].iloc[window:].tolist()

# Next Year: Mid EPS Scenarios
next_yr_mid_bv = adm.loc['Next Year Mid: BV'].iloc[window:]
next_yr_mid_lt_debt = adm.loc['Next Year Mid: LT Debt'].iloc[window:].tolist()
next_yr_mid_st_debt = adm.loc['Next Year Mid: ST Debt'].iloc[window:].tolist()

# Next Year: High EPS Scenarios
next_yr_high_bv = adm.loc['Next Year High: BV'].iloc[window:]
next_yr_high_lt_debt = adm.loc['Next Year High: LT Debt'].iloc[window:].tolist()
next_yr_high_st_debt = adm.loc['Next Year High: ST Debt'].iloc[window:].tolist()

# Next Year: Target Scenarios
target_bv = adm.loc['Target Structure: BV'].iloc[window:]
target_lt_debt = adm.loc['Target Structure: LT Debt'].iloc[window:].tolist()
target_st_debt = adm.loc['Target Structure: ST Debt'].iloc[window:].tolist()


In [ ]:
# Create dates
dates_str = ['2015','2016','2017','2018','2019','2020P',' ','2020 Low EPS','2020 Midpoint EPS','2020 High EPS',' ','Target Structure']

In [ ]:
# Create three variables which are put into the chart: equity book value, lt debt, and st debt

equity_bv_append =[plan_equity[0], 0, next_yr_low_bv[0],next_yr_mid_bv[0],next_yr_high_bv[0],0,target_bv[0]]
equity_bv_var = equity_bv + equity_bv_append
equity_bv_var = [ '%.2f' % elem for elem in equity_bv_var ] # Rounds each number to two decimals

lt_debt_append =[plan_lt_debt[0], 0, next_yr_low_lt_debt[0],next_yr_mid_lt_debt[0],next_yr_high_lt_debt[0],0,target_lt_debt[0]]
lt_debt_var = equity_bv + equity_bv_append
lt_debt_var = [ '%.2f' % elem for elem in lt_debt_var ] # Rounds each number to two decimals

st_debt_append =[plan_st_debt[0], 0,next_yr_low_lt_debt[0],next_yr_mid_st_debt[0],next_yr_high_st_debt[0],0,target_st_debt[0]]
st_debt_var = equity_bv + equity_bv_append
st_debt_var = [ '%.2f' % elem for elem in st_debt_var ] # Rounds each number to two decimals

In [ ]:
# Create chart_data
chart_data = CategoryChartData()

# This updates the dates (i.e. categories)
chart_data.categories = dates_str

# Update the actual data
chart_data.add_series('Equity (Book Value)', equity_bv_var)
chart_data.add_series('LT Debt', lt_debt_var)
chart_data.add_series('ST Debt', st_debt_var)


# Replace existing data
chart.replace_data(chart_data)
chart.has_major_gridlines = False

### Slide 5: S&P Limit vs Actual Leverage

In [ ]:
# The code below shows the idx values for each placeholder
slide = prs.slides[4]
for shape in slide.shapes:
     if shape.is_placeholder:
         phf = shape.placeholder_format
         print('%d, %s' % (phf.idx, phf.type))

In [ ]:
# The code below is needed to access the chart on the fourth slide (i.e. prs.slides[3])
slide = prs.slides[4]
chart = slide.shapes[4].chart

chart_data = CategoryChartData()

In [ ]:
window = -8

# Create variables for chart

sp_lev = adm.loc['SP Actual Leverage'].iloc[window:]
sp_plan = adm.loc['SP Plan Leverage'].iloc[window:]

# Need to create different window length because sp_leverage will have length of 12 (8 quarters of historical and 
# 4 quarters of Plan) compared to limit which only has 8 quarters because it's not getting added to anything

window = -12 
sp_limit = adm.loc['SP Limit'].iloc[window:]

# Combine plan and actual leverage numbers
sp_lev = sp_lev.append(sp_plan[-4:])

# TEST DATA TO ASSESS FUNCTIONALITY
sp_lev[-1] = 1

In [ ]:
# Create chart_data
chart_data = CategoryChartData()

# This updates the dates (i.e. categories)
year = ['2018','2019','2020 Plan']
dates_str = ['Q1','Q2','Q3','Q4','Q1','Q2','Q3','Q4','Q1','Q2','Q3','Q4']
chart_data.categories = dates_str

# Update the actual data
chart_data.add_series('Actual Leverage', sp_limit) # Not sure why but I had to put 'sp limit' in for 'Actual Leverage'. This is reversed of how I'd expect it to be
chart_data.add_series('Limit', sp_lev)


# Replace existing data
chart.replace_data(chart_data)
chart.has_major_gridlines = False

**Slide 6: Rating Agencies Leverage Scenarios Based off Buybacks**

In [ ]:
# The code below shows the idx values for each placeholder
slide = prs.slides[5]
for shape in slide.shapes:
     if shape.is_placeholder:
         phf = shape.placeholder_format
         print('%d, %s' % (phf.idx, phf.type))

In [ ]:
# The code below is needed to access the chart on the sixth slide (i.e. prs.slides[5])
slide = prs.slides[5]
chart = slide.shapes[0].chart

chart_data = CategoryChartData()

In [ ]:
window = -5

# Create S&P variables for chart
sp_100 = adm.loc['SP $100M Buyback'].iloc[window:] # NOTE, THE NEGATIVE SIGN BEFORE WINDOW.
sp_500 = adm.loc['SP $500M Buyback'].iloc[window:]
sp_1000 = adm.loc['SP $1,000 Buyback'].iloc[window:]
sp_limit = adm.loc['SP Lev Scenario Limit'].iloc[window:]

# Create Moody's variables for chart
m_100 = adm.loc['Moodys $100M Buyback'].iloc[window:] # NOTE, THE NEGATIVE SIGN BEFORE WINDOW.
m_500 = adm.loc['Moodys $500M Buyback'].iloc[window:]
m_1000 = adm.loc['Moodys $1,000 Buyback'].iloc[window:]
m_limit = adm.loc['Moodys Lev Scenario Limit'].iloc[window:]

# Create Fitch variables for chart
f_100 = adm.loc['Fitch $100M Buyback'].iloc[window:] # NOTE, THE NEGATIVE SIGN BEFORE WINDOW.
f_500 = adm.loc['Fitch $500M Buyback'].iloc[window:]
f_1000 = adm.loc['Fitch $1,000 Buyback'].iloc[window:]
f_limit = adm.loc['Fitch Lev Scenario Limit'].iloc[window:]

In [ ]:
# Create chart_data
chart = slide.shapes[0].chart
chart_data = CategoryChartData()

# This updates the dates (i.e. categories)
dates_str = ['Q4F','Q1','Q2','Q3','Q4']


################################################### Update the S&P chart
# Update chart shpae (this updates which chart on the slide is being updated)
chart = slide.shapes[0].chart
chart_data = CategoryChartData()

chart_data.add_series('$100M Buyback', sp_100) 
chart_data.add_series('$500M Buyback', sp_500)
chart_data.add_series('$1,000M Buyback', sp_1000)
chart_data.add_series('Limit', sp_limit)
chart_data.categories = dates_str

# Replace existing data for S&P chart
chart.replace_data(chart_data)
chart.has_major_gridlines = False


################################################### Update the Moody's chart
# Update chart shpae (this updates which chart on the slide is being updated)
chart = slide.shapes[2].chart
chart_data = CategoryChartData()

chart_data.add_series('$100M Buyback', m_100) 
chart_data.add_series('$500M Buyback', m_500)
chart_data.add_series('$1,000M Buyback', m_1000)
chart_data.add_series('Limit', m_limit)
chart_data.categories = dates_str

# Replace existing data for Moody's chart
chart.replace_data(chart_data)
chart.has_major_gridlines = False

################################################### Update the Fitch chart
# Update chart shpae (this updates which chart on the slide is being updated)
chart = slide.shapes[3].chart
chart_data = CategoryChartData()

chart_data.add_series('$100M Buyback', f_100) 
chart_data.add_series('$500M Buyback', f_500)
chart_data.add_series('$1,000M Buyback', f_1000)
chart_data.add_series('Limit', f_limit)
chart_data.categories = dates_str

# Replace existing data
chart.replace_data(chart_data)
chart.has_major_gridlines = False

# Replace existing data for Fitch chart
chart.replace_data(chart_data)
chart.has_major_gridlines = False

**Slide 7: Rating Agencies Secondary Ratios (i.e FFO / Debt, RCF / Net Debt, RCF**

In [ ]:
# The code below shows the idx values for each placeholder
slide = prs.slides[6]
for shape in slide.shapes:
     if shape.is_placeholder:
         phf = shape.placeholder_format
         print('%d, %s' % (phf.idx, phf.type))

In [ ]:
# The code below is needed to access the chart on the sixth slide (i.e. prs.slides[5])
slide = prs.slides[6]
chart = slide.shapes[0].chart

chart_data = CategoryChartData()

In [ ]:
window = -5

# Create S&P variables for chart
sp_100 = adm.loc['SP $100M Buyback - FFO'].iloc[window:] # NOTE, THE NEGATIVE SIGN BEFORE WINDOW.
sp_500 = adm.loc['SP $500M Buyback - FFO'].iloc[window:]
sp_1000 = adm.loc['SP $1,000 Buyback - FFO'].iloc[window:]
sp_limit = adm.loc['SP FFO Limit'].iloc[window:]

# Create Moody's variables for chart
m_100 = adm.loc['Moodys $100M Buyback - RCF'].iloc[window:] # NOTE, THE NEGATIVE SIGN BEFORE WINDOW.
m_500 = adm.loc['Moodys $500M Buyback - RCF'].iloc[window:]
m_1000 = adm.loc['Moodys $1,000 Buyback - RCF'].iloc[window:]
m_limit = adm.loc['Moodys RCF Debt Limit'].iloc[window:]

# Create Fitch variables for chart
f_100 = adm.loc['Fitch $100M Buyback - RCF'].iloc[window:] # NOTE, THE NEGATIVE SIGN BEFORE WINDOW.
f_500 = adm.loc['Fitch $500M Buyback - RCF'].iloc[window:]
f_1000 = adm.loc['Fitch $1,000 Buyback - RCF'].iloc[window:]
f_limit = adm.loc['Fitch Gross Debt Limit'].iloc[window:]

In [ ]:
# Create chart_data
chart = slide.shapes[0].chart
chart_data = CategoryChartData()

# This updates the dates (i.e. categories)
dates_str = ['Q4F','Q1','Q2','Q3','Q4']


################################################### Update the S&P chart
# Update chart shpae (this updates which chart on the slide is being updated)
chart = slide.shapes[0].chart
chart_data = CategoryChartData()

chart_data.add_series('$100M Buyback', sp_100) 
chart_data.add_series('$500M Buyback', sp_500)
chart_data.add_series('$1,000M Buyback', sp_1000)
chart_data.add_series('Limit', sp_limit)
chart_data.categories = dates_str

# Replace existing data for S&P chart
chart.replace_data(chart_data)
chart.has_major_gridlines = False


################################################### Update the Moody's chart
# Update chart shpae (this updates which chart on the slide is being updated)
chart = slide.shapes[2].chart
chart_data = CategoryChartData()

chart_data.add_series('$100M Buyback', m_100) 
chart_data.add_series('$500M Buyback', m_500)
chart_data.add_series('$1,000M Buyback', m_1000)
chart_data.add_series('Limit', m_limit)
chart_data.categories = dates_str

# Replace existing data for Moody's chart
chart.replace_data(chart_data)
chart.has_major_gridlines = False

################################################### Update the Fitch chart
# Update chart shpae (this updates which chart on the slide is being updated)
chart = slide.shapes[3].chart
chart_data = CategoryChartData()

chart_data.add_series('$100M Buyback', f_100) 
chart_data.add_series('$500M Buyback', f_500)
chart_data.add_series('$1,000M Buyback', f_1000)
chart_data.add_series('Limit', f_limit)
chart_data.categories = dates_str

# Replace existing data
chart.replace_data(chart_data)
chart.has_major_gridlines = False

# Replace existing data for Fitch chart
chart.replace_data(chart_data)
chart.has_major_gridlines = False

**Save General Slides**

In [ ]:
prs.save('General_Slides - OUTPUT.pptx')

In [ ]:
# The code below shows the idx values for each placeholder
slide = prs.slides[1]

for shape in slide.shapes:
     if shape.is_placeholder:
         phf = shape.placeholder_format
         print('%d, %s' % (phf.idx, phf.type))

**Slide 2: S&P Plan Leverage**

In [ ]:
# The code below is needed to access the chart on the second slide (i.e. prs.slides[1])
slide = prs.slides[1] 
chart = slide.shapes[1].chart # Not sure why choosing shapes[1] works but it does work

chart_data = CategoryChartData()

In [ ]:
# Create window for lookback period - this determines how many historical periods are included (i.e. -4 means 4 historical periods)
window = -4

# Create variables to be place into chart
sp_plan = adm.loc['SP Plan Leverage'].iloc[window:]
sp_actual = adm.loc['SP Actual Leverage'].iloc[window:]
sp_limit = adm.loc['SP Limit'].iloc[window:]

# Create dates
dates_str = adm.columns.astype(str)
dates_str = dates_str[window:]


# TEST DATA ---- CAN DELETE
#sp_plan = [2,2,2,2]

In [ ]:
# Create chart_data
chart_data = CategoryChartData()

# This updates the dates (i.e. categories)
chart_data.categories = dates_str

# Update the actual data
chart_data.add_series('2019 Plan', (sp_plan))
chart_data.add_series('10 + 2F', sp_actual)
chart_data.add_series('Limit', sp_limit)

# Replace existing data
chart.replace_data(chart_data)
chart.has_major_gridlines = False

**Slide 3: FCF, Net M&A Share Repurchases, Dividends**

In [ ]:
# The code below shows the idx values for each placeholder
slide = prs.slides[2]

for shape in slide.shapes:
     if shape.is_placeholder:
         phf = shape.placeholder_format
         print('%d, %s' % (phf.idx, phf.type))

In [ ]:
# The code below is needed to access the chart on the third slide (i.e. prs.slides[2])
slide = prs.slides[2]
chart = slide.shapes[1].chart

chart_data = CategoryChartData()

In [ ]:
# Create window for lookback period - this determines how many historical periods are included (i.e. -4 means 4 historical periods)
window = -25

# Create variables to be place into chart
fcf = adm.loc['FCF'].iloc[window:]
net_ma = adm.loc['Net M&A/Divestitures'].iloc[window:]
share_repurch = adm.loc['Share Repurchases'].iloc[window:]
divs = adm.loc['Dividends'].iloc[window:]
fcf_util = adm.loc['Net FCF Utilization'].iloc[window:]

# Drop the NaNs
fcf = fcf[~np.isnan(fcf)].tolist()
net_ma = net_ma[~np.isnan(net_ma)].tolist()
share_repurch = share_repurch[~np.isnan(share_repurch)].tolist()
divs = divs[~np.isnan(divs)].tolist()
fcf_util = fcf_util[~np.isnan(fcf_util)].tolist()

# Create date
date = pd.Series(['2013','2014','2015','2016','2017','2018','2019'])

In [ ]:
# Create chart_data
chart_data = CategoryChartData()

# This updates the dates (i.e. categories)
chart_data.categories = date

# Update the actual data
chart_data.add_series('FCF', fcf)
chart_data.add_series('Net M&A/Divestitures', net_ma)
chart_data.add_series('Share Repurchases', share_repurch)
chart_data.add_series('Dividends', divs)
chart_data.add_series('Net FCF Utilization', fcf_util)

# Replace existing data
chart.replace_data(chart_data)
chart.has_major_gridlines = False

**Slide 4: Table**

In [ ]:
# The code below shows the idx values for each placeholder
for shape in slide.shapes:
     if shape.is_placeholder:
         phf = shape.placeholder_format
         print('%d, %s' % (phf.idx, phf.type))

In [ ]:
# The code below is needed to access the chart on the fourth slide (i.e. prs.slides[3])
slide = prs.slides[3]
chart = slide.shapes[0].chart

chart_data = CategoryChartData()

In [ ]:
# Create window for lookback period - this determines how many historical periods are included (i.e. -4 means 4 historical periods)
window = -19

# Create data for each variable 
# Historical actual values
equity_bv = adm.loc['Equity (book value)'].iloc[window:] # NOTE, THE NEGATIVE SIGN BEFORE WINDOW. See note above
lt_debt = adm.loc['LT Debt'].iloc[window:]
st_debt = adm.loc['ST Debt'].iloc[window:]

# Drop the NaNs
equity_bv = equity_bv[~np.isnan(equity_bv)].tolist()
lt_debt = lt_debt[~np.isnan(lt_debt)].tolist()
st_debt = st_debt[~np.isnan(st_debt)].tolist()

# Update lookback period
window = -1

# Plan Numbers
plan_equity = adm.loc['Plan: Equity (book value)'].iloc[window:]
plan_lt_debt = adm.loc['Plan: LT Debt'].iloc[window:].tolist()
plan_st_debt = adm.loc['Plan: ST Debt'].iloc[window:].tolist()

# Next Year: Low EPS Scenarios
next_yr_low_bv = adm.loc['Next Year Low: BV'].iloc[window:]
next_yr_low_lt_debt = adm.loc['Next Year Low: LT Debt'].iloc[window:].tolist()
next_yr_low_st_debt = adm.loc['Next Year Low: ST Debt'].iloc[window:].tolist()

# Next Year: Mid EPS Scenarios
next_yr_mid_bv = adm.loc['Next Year Mid: BV'].iloc[window:]
next_yr_mid_lt_debt = adm.loc['Next Year Mid: LT Debt'].iloc[window:].tolist()
next_yr_mid_st_debt = adm.loc['Next Year Mid: ST Debt'].iloc[window:].tolist()

# Next Year: High EPS Scenarios
next_yr_high_bv = adm.loc['Next Year High: BV'].iloc[window:]
next_yr_high_lt_debt = adm.loc['Next Year High: LT Debt'].iloc[window:].tolist()
next_yr_high_st_debt = adm.loc['Next Year High: ST Debt'].iloc[window:].tolist()

# Next Year: Target Scenarios
target_bv = adm.loc['Target Structure: BV'].iloc[window:]
target_lt_debt = adm.loc['Target Structure: LT Debt'].iloc[window:].tolist()
target_st_debt = adm.loc['Target Structure: ST Debt'].iloc[window:].tolist()


In [ ]:
# Create dates
dates_str = ['2015','2016','2017','2018','2019','2020P',' ','2020 Low EPS','2020 Midpoint EPS','2020 High EPS',' ','Target Structure']

In [ ]:
# Create three variables which are put into the chart: equity book value, lt debt, and st debt

equity_bv_append =[plan_equity[0], 0, next_yr_low_bv[0],next_yr_mid_bv[0],next_yr_high_bv[0],0,target_bv[0]]
equity_bv_var = equity_bv + equity_bv_append
equity_bv_var = [ '%.2f' % elem for elem in equity_bv_var ] # Rounds each number to two decimals

lt_debt_append =[plan_lt_debt[0], 0, next_yr_low_lt_debt[0],next_yr_mid_lt_debt[0],next_yr_high_lt_debt[0],0,target_lt_debt[0]]
lt_debt_var = equity_bv + equity_bv_append
lt_debt_var = [ '%.2f' % elem for elem in lt_debt_var ] # Rounds each number to two decimals

st_debt_append =[plan_st_debt[0], 0,next_yr_low_lt_debt[0],next_yr_mid_st_debt[0],next_yr_high_st_debt[0],0,target_st_debt[0]]
st_debt_var = equity_bv + equity_bv_append
st_debt_var = [ '%.2f' % elem for elem in st_debt_var ] # Rounds each number to two decimals

In [ ]:
# Create chart_data
chart_data = CategoryChartData()

# This updates the dates (i.e. categories)
chart_data.categories = dates_str

# Update the actual data
chart_data.add_series('Equity (Book Value)', equity_bv_var)
chart_data.add_series('LT Debt', lt_debt_var)
chart_data.add_series('ST Debt', st_debt_var)


# Replace existing data
chart.replace_data(chart_data)
chart.has_major_gridlines = False

### Slide 5: S&P Limit vs Actual Leverage

In [ ]:
# The code below shows the idx values for each placeholder
slide = prs.slides[4]
for shape in slide.shapes:
     if shape.is_placeholder:
         phf = shape.placeholder_format
         print('%d, %s' % (phf.idx, phf.type))

In [ ]:
# The code below is needed to access the chart on the fourth slide (i.e. prs.slides[3])
slide = prs.slides[4]
chart = slide.shapes[4].chart

chart_data = CategoryChartData()

In [ ]:
window = -8

# Create variables for chart

sp_lev = adm.loc['SP Actual Leverage'].iloc[window:]
sp_plan = adm.loc['SP Plan Leverage'].iloc[window:]

# Need to create different window length because sp_leverage will have length of 12 (8 quarters of historical and 
# 4 quarters of Plan) compared to limit which only has 8 quarters because it's not getting added to anything

window = -12 
sp_limit = adm.loc['SP Limit'].iloc[window:]

# Combine plan and actual leverage numbers
sp_lev = sp_lev.append(sp_plan[-4:])

# TEST DATA TO ASSESS FUNCTIONALITY
sp_lev[-1] = 1

In [ ]:
# Create chart_data
chart_data = CategoryChartData()

# This updates the dates (i.e. categories)
year = ['2018','2019','2020 Plan']
dates_str = ['Q1','Q2','Q3','Q4','Q1','Q2','Q3','Q4','Q1','Q2','Q3','Q4']
chart_data.categories = dates_str

# Update the actual data
chart_data.add_series('Actual Leverage', sp_limit) # Not sure why but I had to put 'sp limit' in for 'Actual Leverage'. This is reversed of how I'd expect it to be
chart_data.add_series('Limit', sp_lev)


# Replace existing data
chart.replace_data(chart_data)
chart.has_major_gridlines = False

**Slide 6: Rating Agencies Leverage Scenarios Based off Buybacks**

In [ ]:
# The code below shows the idx values for each placeholder
slide = prs.slides[5]
for shape in slide.shapes:
     if shape.is_placeholder:
         phf = shape.placeholder_format
         print('%d, %s' % (phf.idx, phf.type))

In [ ]:
# The code below is needed to access the chart on the sixth slide (i.e. prs.slides[5])
slide = prs.slides[5]
chart = slide.shapes[0].chart

chart_data = CategoryChartData()

In [ ]:
window = -5

# Create S&P variables for chart
sp_100 = adm.loc['SP $100M Buyback'].iloc[window:] # NOTE, THE NEGATIVE SIGN BEFORE WINDOW.
sp_500 = adm.loc['SP $500M Buyback'].iloc[window:]
sp_1000 = adm.loc['SP $1,000 Buyback'].iloc[window:]
sp_limit = adm.loc['SP Lev Scenario Limit'].iloc[window:]

# Create Moody's variables for chart
m_100 = adm.loc['Moodys $100M Buyback'].iloc[window:] # NOTE, THE NEGATIVE SIGN BEFORE WINDOW.
m_500 = adm.loc['Moodys $500M Buyback'].iloc[window:]
m_1000 = adm.loc['Moodys $1,000 Buyback'].iloc[window:]
m_limit = adm.loc['Moodys Lev Scenario Limit'].iloc[window:]

# Create Fitch variables for chart
f_100 = adm.loc['Fitch $100M Buyback'].iloc[window:] # NOTE, THE NEGATIVE SIGN BEFORE WINDOW.
f_500 = adm.loc['Fitch $500M Buyback'].iloc[window:]
f_1000 = adm.loc['Fitch $1,000 Buyback'].iloc[window:]
f_limit = adm.loc['Fitch Lev Scenario Limit'].iloc[window:]

In [ ]:
# Create chart_data
chart = slide.shapes[0].chart
chart_data = CategoryChartData()

# This updates the dates (i.e. categories)
dates_str = ['Q4F','Q1','Q2','Q3','Q4']


################################################### Update the S&P chart
# Update chart shpae (this updates which chart on the slide is being updated)
chart = slide.shapes[0].chart
chart_data = CategoryChartData()

chart_data.add_series('$100M Buyback', sp_100) 
chart_data.add_series('$500M Buyback', sp_500)
chart_data.add_series('$1,000M Buyback', sp_1000)
chart_data.add_series('Limit', sp_limit)
chart_data.categories = dates_str

# Replace existing data for S&P chart
chart.replace_data(chart_data)
chart.has_major_gridlines = False


################################################### Update the Moody's chart
# Update chart shpae (this updates which chart on the slide is being updated)
chart = slide.shapes[2].chart
chart_data = CategoryChartData()

chart_data.add_series('$100M Buyback', m_100) 
chart_data.add_series('$500M Buyback', m_500)
chart_data.add_series('$1,000M Buyback', m_1000)
chart_data.add_series('Limit', m_limit)
chart_data.categories = dates_str

# Replace existing data for Moody's chart
chart.replace_data(chart_data)
chart.has_major_gridlines = False

################################################### Update the Fitch chart
# Update chart shpae (this updates which chart on the slide is being updated)
chart = slide.shapes[3].chart
chart_data = CategoryChartData()

chart_data.add_series('$100M Buyback', f_100) 
chart_data.add_series('$500M Buyback', f_500)
chart_data.add_series('$1,000M Buyback', f_1000)
chart_data.add_series('Limit', f_limit)
chart_data.categories = dates_str

# Replace existing data
chart.replace_data(chart_data)
chart.has_major_gridlines = False

# Replace existing data for Fitch chart
chart.replace_data(chart_data)
chart.has_major_gridlines = False

**Slide 7: Rating Agencies Secondary Ratios (i.e FFO / Debt, RCF / Net Debt, RCF**

In [ ]:
# The code below shows the idx values for each placeholder
slide = prs.slides[6]
for shape in slide.shapes:
     if shape.is_placeholder:
         phf = shape.placeholder_format
         print('%d, %s' % (phf.idx, phf.type))

In [ ]:
# The code below is needed to access the chart on the sixth slide (i.e. prs.slides[5])
slide = prs.slides[6]
chart = slide.shapes[0].chart

chart_data = CategoryChartData()

In [ ]:
window = -5

# Create S&P variables for chart
sp_100 = adm.loc['SP $100M Buyback - FFO'].iloc[window:] # NOTE, THE NEGATIVE SIGN BEFORE WINDOW.
sp_500 = adm.loc['SP $500M Buyback - FFO'].iloc[window:]
sp_1000 = adm.loc['SP $1,000 Buyback - FFO'].iloc[window:]
sp_limit = adm.loc['SP FFO Limit'].iloc[window:]

# Create Moody's variables for chart
m_100 = adm.loc['Moodys $100M Buyback - RCF'].iloc[window:] # NOTE, THE NEGATIVE SIGN BEFORE WINDOW.
m_500 = adm.loc['Moodys $500M Buyback - RCF'].iloc[window:]
m_1000 = adm.loc['Moodys $1,000 Buyback - RCF'].iloc[window:]
m_limit = adm.loc['Moodys RCF Debt Limit'].iloc[window:]

# Create Fitch variables for chart
f_100 = adm.loc['Fitch $100M Buyback - RCF'].iloc[window:] # NOTE, THE NEGATIVE SIGN BEFORE WINDOW.
f_500 = adm.loc['Fitch $500M Buyback - RCF'].iloc[window:]
f_1000 = adm.loc['Fitch $1,000 Buyback - RCF'].iloc[window:]
f_limit = adm.loc['Fitch Gross Debt Limit'].iloc[window:]

In [ ]:
# Create chart_data
chart = slide.shapes[0].chart
chart_data = CategoryChartData()

# This updates the dates (i.e. categories)
dates_str = ['Q4F','Q1','Q2','Q3','Q4']


################################################### Update the S&P chart
# Update chart shpae (this updates which chart on the slide is being updated)
chart = slide.shapes[0].chart
chart_data = CategoryChartData()

chart_data.add_series('$100M Buyback', sp_100) 
chart_data.add_series('$500M Buyback', sp_500)
chart_data.add_series('$1,000M Buyback', sp_1000)
chart_data.add_series('Limit', sp_limit)
chart_data.categories = dates_str

# Replace existing data for S&P chart
chart.replace_data(chart_data)
chart.has_major_gridlines = False


################################################### Update the Moody's chart
# Update chart shpae (this updates which chart on the slide is being updated)
chart = slide.shapes[2].chart
chart_data = CategoryChartData()

chart_data.add_series('$100M Buyback', m_100) 
chart_data.add_series('$500M Buyback', m_500)
chart_data.add_series('$1,000M Buyback', m_1000)
chart_data.add_series('Limit', m_limit)
chart_data.categories = dates_str

# Replace existing data for Moody's chart
chart.replace_data(chart_data)
chart.has_major_gridlines = False

################################################### Update the Fitch chart
# Update chart shpae (this updates which chart on the slide is being updated)
chart = slide.shapes[3].chart
chart_data = CategoryChartData()

chart_data.add_series('$100M Buyback', f_100) 
chart_data.add_series('$500M Buyback', f_500)
chart_data.add_series('$1,000M Buyback', f_1000)
chart_data.add_series('Limit', f_limit)
chart_data.categories = dates_str

# Replace existing data
chart.replace_data(chart_data)
chart.has_major_gridlines = False

# Replace existing data for Fitch chart
chart.replace_data(chart_data)
chart.has_major_gridlines = False

**Save General Slides**

In [ ]:
prs.save('General_Slides - OUTPUT.pptx')

# Generate General Slides

In [ ]:
# This loads the MASTER presentation
prs = Presentation('General_Slides - MASTER2.pptx')

In [ ]:
# The code below shows the idx values for each placeholder
slide = prs.slides[1]

for shape in slide.shapes:
     if shape.is_placeholder:
         phf = shape.placeholder_format
         print('%d, %s' % (phf.idx, phf.type))

**Slide 2: S&P Plan Leverage**

In [ ]:
# The code below is needed to access the chart on the second slide (i.e. prs.slides[1])
slide = prs.slides[1] 
chart = slide.shapes[1].chart # Not sure why choosing shapes[1] works but it does work

chart_data = CategoryChartData()

In [ ]:
# Create window for lookback period - this determines how many historical periods are included (i.e. -4 means 4 historical periods)
window = -4

# Create variables to be place into chart
sp_plan = adm.loc['SP Plan Leverage'].iloc[window:]
sp_actual = adm.loc['SP Actual Leverage'].iloc[window:]
sp_limit = adm.loc['SP Limit'].iloc[window:]

# Create dates
dates_str = adm.columns.astype(str)
dates_str = dates_str[window:]


# TEST DATA ---- CAN DELETE
#sp_plan = [2,2,2,2]

In [ ]:
# Create chart_data
chart_data = CategoryChartData()

# This updates the dates (i.e. categories)
chart_data.categories = dates_str

# Update the actual data
chart_data.add_series('2019 Plan', (sp_plan))
chart_data.add_series('10 + 2F', sp_actual)
chart_data.add_series('Limit', sp_limit)

# Replace existing data
chart.replace_data(chart_data)
chart.has_major_gridlines = False

**Slide 3: FCF, Net M&A Share Repurchases, Dividends**

In [ ]:
# The code below shows the idx values for each placeholder
slide = prs.slides[2]

for shape in slide.shapes:
     if shape.is_placeholder:
         phf = shape.placeholder_format
         print('%d, %s' % (phf.idx, phf.type))

In [ ]:
# The code below is needed to access the chart on the third slide (i.e. prs.slides[2])
slide = prs.slides[2]
chart = slide.shapes[1].chart

chart_data = CategoryChartData()

In [ ]:
# Create window for lookback period - this determines how many historical periods are included (i.e. -4 means 4 historical periods)
window = -25

# Create variables to be place into chart
fcf = adm.loc['FCF'].iloc[window:]
net_ma = adm.loc['Net M&A/Divestitures'].iloc[window:]
share_repurch = adm.loc['Share Repurchases'].iloc[window:]
divs = adm.loc['Dividends'].iloc[window:]
fcf_util = adm.loc['Net FCF Utilization'].iloc[window:]

# Drop the NaNs
fcf = fcf[~np.isnan(fcf)].tolist()
net_ma = net_ma[~np.isnan(net_ma)].tolist()
share_repurch = share_repurch[~np.isnan(share_repurch)].tolist()
divs = divs[~np.isnan(divs)].tolist()
fcf_util = fcf_util[~np.isnan(fcf_util)].tolist()

# Create date
date = pd.Series(['2013','2014','2015','2016','2017','2018','2019'])

In [ ]:
# Create chart_data
chart_data = CategoryChartData()

# This updates the dates (i.e. categories)
chart_data.categories = date

# Update the actual data
chart_data.add_series('FCF', fcf)
chart_data.add_series('Net M&A/Divestitures', net_ma)
chart_data.add_series('Share Repurchases', share_repurch)
chart_data.add_series('Dividends', divs)
chart_data.add_series('Net FCF Utilization', fcf_util)

# Replace existing data
chart.replace_data(chart_data)
chart.has_major_gridlines = False

**Slide 4: Capital Plan Recommendation: Graph of Equity Book Value, LT Debt, and ST Debt**

In [ ]:
# The code below shows the idx values for each placeholder
for shape in slide.shapes:
     if shape.is_placeholder:
         phf = shape.placeholder_format
         print('%d, %s' % (phf.idx, phf.type))

In [ ]:
# The code below is needed to access the chart on the fourth slide (i.e. prs.slides[3])
slide = prs.slides[3]
chart = slide.shapes[0].chart

chart_data = CategoryChartData()

In [ ]:
# Create window for lookback period - this determines how many historical periods are included (i.e. -4 means 4 historical periods)
window = -19

# Create data for each variable 
# Historical actual values
equity_bv = adm.loc['Equity (book value)'].iloc[window:] # NOTE, THE NEGATIVE SIGN BEFORE WINDOW. See note above
lt_debt = adm.loc['LT Debt'].iloc[window:]
st_debt = adm.loc['ST Debt'].iloc[window:]

# Drop the NaNs
equity_bv = equity_bv[~np.isnan(equity_bv)].tolist()
lt_debt = lt_debt[~np.isnan(lt_debt)].tolist()
st_debt = st_debt[~np.isnan(st_debt)].tolist()

# Update lookback period
window = -1

# Plan Numbers
plan_equity = adm.loc['Plan: Equity (book value)'].iloc[window:]
plan_lt_debt = adm.loc['Plan: LT Debt'].iloc[window:].tolist()
plan_st_debt = adm.loc['Plan: ST Debt'].iloc[window:].tolist()

# Next Year: Low EPS Scenarios
next_yr_low_bv = adm.loc['Next Year Low: BV'].iloc[window:]
next_yr_low_lt_debt = adm.loc['Next Year Low: LT Debt'].iloc[window:].tolist()
next_yr_low_st_debt = adm.loc['Next Year Low: ST Debt'].iloc[window:].tolist()

# Next Year: Mid EPS Scenarios
next_yr_mid_bv = adm.loc['Next Year Mid: BV'].iloc[window:]
next_yr_mid_lt_debt = adm.loc['Next Year Mid: LT Debt'].iloc[window:].tolist()
next_yr_mid_st_debt = adm.loc['Next Year Mid: ST Debt'].iloc[window:].tolist()

# Next Year: High EPS Scenarios
next_yr_high_bv = adm.loc['Next Year High: BV'].iloc[window:]
next_yr_high_lt_debt = adm.loc['Next Year High: LT Debt'].iloc[window:].tolist()
next_yr_high_st_debt = adm.loc['Next Year High: ST Debt'].iloc[window:].tolist()

# Next Year: Target Scenarios
target_bv = adm.loc['Target Structure: BV'].iloc[window:]
target_lt_debt = adm.loc['Target Structure: LT Debt'].iloc[window:].tolist()
target_st_debt = adm.loc['Target Structure: ST Debt'].iloc[window:].tolist()


In [ ]:
# Create dates
dates_str = ['2015','2016','2017','2018','2019','2020P',' ','2020 Low EPS','2020 Midpoint EPS','2020 High EPS',' ','Target Structure']

In [ ]:
# Create three variables which are put into the chart: equity book value, lt debt, and st debt

equity_bv_append =[plan_equity[0], 0, next_yr_low_bv[0],next_yr_mid_bv[0],next_yr_high_bv[0],0,target_bv[0]]
equity_bv_var = equity_bv + equity_bv_append
equity_bv_var = [ '%.2f' % elem for elem in equity_bv_var ] # Rounds each number to two decimals

lt_debt_append =[plan_lt_debt[0], 0, next_yr_low_lt_debt[0],next_yr_mid_lt_debt[0],next_yr_high_lt_debt[0],0,target_lt_debt[0]]
lt_debt_var = equity_bv + equity_bv_append
lt_debt_var = [ '%.2f' % elem for elem in lt_debt_var ] # Rounds each number to two decimals

st_debt_append =[plan_st_debt[0], 0,next_yr_low_lt_debt[0],next_yr_mid_st_debt[0],next_yr_high_st_debt[0],0,target_st_debt[0]]
st_debt_var = equity_bv + equity_bv_append
st_debt_var = [ '%.2f' % elem for elem in st_debt_var ] # Rounds each number to two decimals

In [ ]:
# Create chart_data
chart_data = CategoryChartData()

# This updates the dates (i.e. categories)
chart_data.categories = dates_str

# Update the actual data
chart_data.add_series('Equity (Book Value)', equity_bv_var)
chart_data.add_series('LT Debt', lt_debt_var)
chart_data.add_series('ST Debt', st_debt_var)


# Replace existing data
chart.replace_data(chart_data)
chart.has_major_gridlines = False

### Slide 5: S&P Limit vs Actual Leverage

In [ ]:
# The code below shows the idx values for each placeholder
slide = prs.slides[4]
for shape in slide.shapes:
     if shape.is_placeholder:
         phf = shape.placeholder_format
         print('%d, %s' % (phf.idx, phf.type))

In [ ]:
# The code below is needed to access the chart on the fourth slide (i.e. prs.slides[3])
slide = prs.slides[4]
chart = slide.shapes[4].chart

chart_data = CategoryChartData()

In [ ]:
window = -8

# Create variables for chart

sp_lev = adm.loc['SP Actual Leverage'].iloc[window:]
sp_plan = adm.loc['SP Plan Leverage'].iloc[window:]

# Need to create different window length because sp_leverage will have length of 12 (8 quarters of historical and 
# 4 quarters of Plan) compared to limit which only has 8 quarters because it's not getting added to anything

window = -12 
sp_limit = adm.loc['SP Limit'].iloc[window:]

# Combine plan and actual leverage numbers
sp_lev = sp_lev.append(sp_plan[-4:])

# TEST DATA TO ASSESS FUNCTIONALITY
sp_lev[-1] = 1

In [ ]:
# Create chart_data
chart_data = CategoryChartData()

# This updates the dates (i.e. categories)
year = ['2018','2019','2020 Plan']
dates_str = ['Q1','Q2','Q3','Q4','Q1','Q2','Q3','Q4','Q1','Q2','Q3','Q4']
chart_data.categories = dates_str

# Update the actual data
chart_data.add_series('Actual Leverage', sp_limit) # Not sure why but I had to put 'sp limit' in for 'Actual Leverage'. This is reversed of how I'd expect it to be
chart_data.add_series('Limit', sp_lev)


# Replace existing data
chart.replace_data(chart_data)
chart.has_major_gridlines = False

**Slide 6: Rating Agencies Leverage Scenarios Based off Buybacks**

In [ ]:
# The code below shows the idx values for each placeholder
slide = prs.slides[5]
for shape in slide.shapes:
     if shape.is_placeholder:
         phf = shape.placeholder_format
         print('%d, %s' % (phf.idx, phf.type))

In [ ]:
# The code below is needed to access the chart on the sixth slide (i.e. prs.slides[5])
slide = prs.slides[5]
chart = slide.shapes[0].chart

chart_data = CategoryChartData()

In [ ]:
window = -5

# Create S&P variables for chart
sp_100 = adm.loc['SP $100M Buyback'].iloc[window:] # NOTE, THE NEGATIVE SIGN BEFORE WINDOW.
sp_500 = adm.loc['SP $500M Buyback'].iloc[window:]
sp_1000 = adm.loc['SP $1,000 Buyback'].iloc[window:]
sp_limit = adm.loc['SP Lev Scenario Limit'].iloc[window:]

# Create Moody's variables for chart
m_100 = adm.loc['Moodys $100M Buyback'].iloc[window:] # NOTE, THE NEGATIVE SIGN BEFORE WINDOW.
m_500 = adm.loc['Moodys $500M Buyback'].iloc[window:]
m_1000 = adm.loc['Moodys $1,000 Buyback'].iloc[window:]
m_limit = adm.loc['Moodys Lev Scenario Limit'].iloc[window:]

# Create Fitch variables for chart
f_100 = adm.loc['Fitch $100M Buyback'].iloc[window:] # NOTE, THE NEGATIVE SIGN BEFORE WINDOW.
f_500 = adm.loc['Fitch $500M Buyback'].iloc[window:]
f_1000 = adm.loc['Fitch $1,000 Buyback'].iloc[window:]
f_limit = adm.loc['Fitch Lev Scenario Limit'].iloc[window:]

In [ ]:
# Create chart_data
chart = slide.shapes[0].chart
chart_data = CategoryChartData()

# This updates the dates (i.e. categories)
dates_str = ['Q4F','Q1','Q2','Q3','Q4']


################################################### Update the S&P chart
# Update chart shpae (this updates which chart on the slide is being updated)
chart = slide.shapes[0].chart
chart_data = CategoryChartData()

chart_data.add_series('$100M Buyback', sp_100) 
chart_data.add_series('$500M Buyback', sp_500)
chart_data.add_series('$1,000M Buyback', sp_1000)
chart_data.add_series('Limit', sp_limit)
chart_data.categories = dates_str

# Replace existing data for S&P chart
chart.replace_data(chart_data)
chart.has_major_gridlines = False


################################################### Update the Moody's chart
# Update chart shpae (this updates which chart on the slide is being updated)
chart = slide.shapes[2].chart
chart_data = CategoryChartData()

chart_data.add_series('$100M Buyback', m_100) 
chart_data.add_series('$500M Buyback', m_500)
chart_data.add_series('$1,000M Buyback', m_1000)
chart_data.add_series('Limit', m_limit)
chart_data.categories = dates_str

# Replace existing data for Moody's chart
chart.replace_data(chart_data)
chart.has_major_gridlines = False

################################################### Update the Fitch chart
# Update chart shpae (this updates which chart on the slide is being updated)
chart = slide.shapes[3].chart
chart_data = CategoryChartData()

chart_data.add_series('$100M Buyback', f_100) 
chart_data.add_series('$500M Buyback', f_500)
chart_data.add_series('$1,000M Buyback', f_1000)
chart_data.add_series('Limit', f_limit)
chart_data.categories = dates_str

# Replace existing data
chart.replace_data(chart_data)
chart.has_major_gridlines = False

# Replace existing data for Fitch chart
chart.replace_data(chart_data)
chart.has_major_gridlines = False

**Slide 7: Rating Agencies Secondary Ratios (i.e FFO / Debt, RCF / Net Debt, RCF**

In [ ]:
# The code below shows the idx values for each placeholder
slide = prs.slides[6]
for shape in slide.shapes:
     if shape.is_placeholder:
         phf = shape.placeholder_format
         print('%d, %s' % (phf.idx, phf.type))

In [ ]:
# The code below is needed to access the chart on the sixth slide (i.e. prs.slides[5])
slide = prs.slides[6]
chart = slide.shapes[0].chart

chart_data = CategoryChartData()

In [ ]:
window = -5

# Create S&P variables for chart
sp_100 = adm.loc['SP $100M Buyback - FFO'].iloc[window:] # NOTE, THE NEGATIVE SIGN BEFORE WINDOW.
sp_500 = adm.loc['SP $500M Buyback - FFO'].iloc[window:]
sp_1000 = adm.loc['SP $1,000 Buyback - FFO'].iloc[window:]
sp_limit = adm.loc['SP FFO Limit'].iloc[window:]

# Create Moody's variables for chart
m_100 = adm.loc['Moodys $100M Buyback - RCF'].iloc[window:] # NOTE, THE NEGATIVE SIGN BEFORE WINDOW.
m_500 = adm.loc['Moodys $500M Buyback - RCF'].iloc[window:]
m_1000 = adm.loc['Moodys $1,000 Buyback - RCF'].iloc[window:]
m_limit = adm.loc['Moodys RCF Debt Limit'].iloc[window:]

# Create Fitch variables for chart
f_100 = adm.loc['Fitch $100M Buyback - RCF'].iloc[window:] # NOTE, THE NEGATIVE SIGN BEFORE WINDOW.
f_500 = adm.loc['Fitch $500M Buyback - RCF'].iloc[window:]
f_1000 = adm.loc['Fitch $1,000 Buyback - RCF'].iloc[window:]
f_limit = adm.loc['Fitch Gross Debt Limit'].iloc[window:]

In [ ]:
# Create chart_data
chart = slide.shapes[0].chart
chart_data = CategoryChartData()

# This updates the dates (i.e. categories)
dates_str = ['Q4F','Q1','Q2','Q3','Q4']


################################################### Update the S&P chart
# Update chart shpae (this updates which chart on the slide is being updated)
chart = slide.shapes[0].chart
chart_data = CategoryChartData()

chart_data.add_series('$100M Buyback', sp_100) 
chart_data.add_series('$500M Buyback', sp_500)
chart_data.add_series('$1,000M Buyback', sp_1000)
chart_data.add_series('Limit', sp_limit)
chart_data.categories = dates_str

# Replace existing data for S&P chart
chart.replace_data(chart_data)
chart.has_major_gridlines = False


################################################### Update the Moody's chart
# Update chart shpae (this updates which chart on the slide is being updated)
chart = slide.shapes[2].chart
chart_data = CategoryChartData()

chart_data.add_series('$100M Buyback', m_100) 
chart_data.add_series('$500M Buyback', m_500)
chart_data.add_series('$1,000M Buyback', m_1000)
chart_data.add_series('Limit', m_limit)
chart_data.categories = dates_str

# Replace existing data for Moody's chart
chart.replace_data(chart_data)
chart.has_major_gridlines = False

################################################### Update the Fitch chart
# Update chart shpae (this updates which chart on the slide is being updated)
chart = slide.shapes[3].chart
chart_data = CategoryChartData()

chart_data.add_series('$100M Buyback', f_100) 
chart_data.add_series('$500M Buyback', f_500)
chart_data.add_series('$1,000M Buyback', f_1000)
chart_data.add_series('Limit', f_limit)
chart_data.categories = dates_str

# Replace existing data
chart.replace_data(chart_data)
chart.has_major_gridlines = False

# Replace existing data for Fitch chart
chart.replace_data(chart_data)
chart.has_major_gridlines = False

**Save General Slides**

In [ ]:
prs.save('General_Slides - OUTPUT.pptx')

In [ ]:
print('test')